# CoRegistery Planet Imagery to LandSat
## Prerequisites
- you need to run the script that converts the float 32 TOA landsat imagery into 16-bit unsigned integers
- you'll need to have a landsat and a planet image for the same location
- both of these images will need to be cloud and Cloud Shadow free
## Functionality
This script works by first retiling the landsat imagery so that each block is a multiple of 16. In this case we choose each block size to be 256 by 256. 

After the landsat image has been retiled then we load the retiled image into  Geo wombat and do the same thing for another landsat then we can register them together and it works

In [2]:
import rasterio
from rasterio.transform import from_origin
import geowombat as gw


def create_tiled_raster(src_filepath, dst_filepath, block_size=256):
    # Open the source file
    with rasterio.open(src_filepath) as src:
        # Copy the profile from the source
        profile = src.profile
        
        # Update the profile to set tiling and block sizes
        profile.update(
            tiled=True,
            blockxsize=block_size,
            blockysize=block_size,
            compress='lzw'  # Optional: Add compression to reduce file size
        )
        
        # Create a new file with the updated profile
        with rasterio.open(dst_filepath, 'w', **profile) as dst:
            # Copy data from the source file to the destination file
            for i in range(1, src.count + 1):
                band_data = src.read(i)
                dst.write(band_data, i)

    print(f'Created a new tiled raster file with block size {block_size}x{block_size} at {dst_filepath}')

def print_block_shapes(filepath):
    # after creating a new block size lets print the new block shape and then attempt to load it with geowombat
    with rasterio.open(dst_filepath) as src:
        # Iterate through each band in the dataset
        for i in range(1, src.count + 1):
            # Access the band
            band = src.read(i)

            # Get the block sizes for the band
            block_shapes = src.block_shapes[i-1]  # block_shapes is zero-indexed in Python

            print(f'Band {i} block size: {block_shapes}')

def open_raster_with_geowombat(filepath):
    with gw.open(filepath) as src:
        print(src)

# Source file path
src_filepath = r"2023-02-14-10-33-43_L8_ID_mqq51_unit16.tif"
# Destination file path
dst_filepath = r"2023-02-14-10-33-43_L8_ID_mqq51_unit16_new_block.tif"
create_tiled_raster(src_filepath, dst_filepath)
print_block_shapes(dst_filepath)
open_raster_with_geowombat(dst_filepath)



Created a new tiled raster file with block size 256x256 at 2023-02-14-10-33-43_L8_ID_mqq51_unit16_new_block.tif
Band 1 block size: (256, 256)
Band 2 block size: (256, 256)
Band 3 block size: (256, 256)
Band 4 block size: (256, 256)
Band 5 block size: (256, 256)
<xarray.DataArray (band: 5, y: 300, x: 299)> Size: 897kB
dask.array<open_rasterio-4adce1dfdb7dbea947ff8c0d9e6ae10b<this-array>, shape=(5, 300, 299), dtype=uint16, chunksize=(5, 256, 256), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int32 20B 1 2 3 4 5
  * x        (x) float64 2kB 6.626e+05 6.626e+05 ... 6.671e+05 6.671e+05
  * y        (y) float64 2kB 5.927e+06 5.927e+06 ... 5.923e+06 5.923e+06
Attributes: (12/13)
    transform:           (15.0, 0.0, 662617.5, 0.0, -15.0, 5927032.5)
    crs:                 32631
    res:                 (15.0, 15.0)
    is_tiled:            0
    nodatavals:          (0.0, 0.0, 0.0, 0.0, 0.0)
    _FillValue:          0.0
    ...                  ...
    offsets:             (0.0,

In [4]:
# Source file path
src_filepath = r"ID_mqq51_sample_int3.tif"
# Destination file path
dst_filepath2 = r"ID_mqq51_sample_int3_new_block.tif"
create_tiled_raster(src_filepath, dst_filepath2)
print_block_shapes(dst_filepath2)
open_raster_with_geowombat(dst_filepath)

Created a new tiled raster file with block size 256x256 at ID_mqq51_sample_int3_new_block.tif
Band 1 block size: (256, 256)
Band 2 block size: (256, 256)
Band 3 block size: (256, 256)
Band 4 block size: (256, 256)
Band 5 block size: (256, 256)
<xarray.DataArray (band: 5, y: 300, x: 299)> Size: 897kB
dask.array<open_rasterio-4adce1dfdb7dbea947ff8c0d9e6ae10b<this-array>, shape=(5, 300, 299), dtype=uint16, chunksize=(5, 256, 256), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int32 20B 1 2 3 4 5
  * x        (x) float64 2kB 6.626e+05 6.626e+05 ... 6.671e+05 6.671e+05
  * y        (y) float64 2kB 5.927e+06 5.927e+06 ... 5.923e+06 5.923e+06
Attributes: (12/13)
    transform:           (15.0, 0.0, 662617.5, 0.0, -15.0, 5927032.5)
    crs:                 32631
    res:                 (15.0, 15.0)
    is_tiled:            0
    nodatavals:          (0.0, 0.0, 0.0, 0.0, 0.0)
    _FillValue:          0.0
    ...                  ...
    offsets:             (0.0, 0.0, 0.0, 0.0, 0.0

# Coregister a landsat to a landsat image

In [5]:
# now lets try to coregister the two files

target_path = r"2023-02-14-10-33-43_L8_ID_mqq51_unit16_new_block.tif"
ref_path = r"ID_mqq51_sample_int3_new_block.tif"

with gw.open(target_path) as target, gw.open(
    ref_path
) as reference:
    target_shifted = gw.coregister(
        target=target,
        reference=reference,
        align_grids = True,
        resamp_alg_deshift='nearest',
        resamp_alg_calc='cubic',
        q=False, #quiet mode (default: False)
        CPUs=1, #  number of CPUs to use during pixel grid equalization (default: None, which means ‘all CPUs available’)
    )
    print(target_shifted)
    target_shifted
    gw.save(target_shifted, 'coregister_2023-02-14-10-33-43_L8_ID_mqq51_new_block.tif', compress='lzw', num_workers=8)

Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00
Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00


Calculating footprint polygon and actual data corner coordinates for reference image...
Bounding box of calculated footprint for reference image:
	(662617.5, 5922562.5, 667087.5, 5927032.5)
Calculating footprint polygon and actual data corner coordinates for image to be shifted...
Bounding box of calculated footprint for image to be shifted:
	(662617.5, 5922562.5, 667087.5, 5927032.5)
Matching window position (X,Y): 664852.5/5924797.5
Detected integer shifts (X/Y):                            0/0
Detected subpixel shifts (X/Y):                           0.016993545913379916/0.1339814765302875
Calculated total shifts in fft pixel units (X/Y):         0.016993545913379916/0.1339814765302875
Calculated total shifts in reference pixel units (X/Y):   0.016993545913379916/0.1339814765302875
Calculated total shifts in target pixel units (X/Y):      0.016993545913379916/0.1339814765302875
Calculated map shifts (X,Y):				  0.25490318867377937/-2.0097221480682492
Calculated absolute shift vector 

Warping progress     |==================================================| 100.0% Complete  => 0:00:00
11:17:42:WARNING:755:io.save:The file coregister_2023-02-14-10-33-43_L8_ID_mqq51_new_block.tif already exists.


Image similarity within the matching window (SSIM before/after correction): 0.5400 => 0.5352
Estimated reliability of the calculated shifts:  82.9 %
Correcting geometric shifts...
<xarray.DataArray 'array-1a6484367fdaaa7063dfefc70141f8c7' (band: 5, y: 300,
                                                            x: 299)> Size: 4MB
dask.array<array, shape=(5, 300, 299), dtype=float64, chunksize=(5, 256, 256), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int32 20B 1 2 3 4 5
  * y        (y) float64 2kB 5.927e+06 5.927e+06 ... 5.923e+06 5.923e+06
  * x        (x) float64 2kB 6.626e+05 6.626e+05 ... 6.671e+05 6.671e+05
Attributes: (12/15)
    transform:           (15.0, 0.0, 662617.5, 0.0, -15.0, 5927032.5)
    crs:                 32631
    res:                 (15.0, 15.0)
    is_tiled:            0
    nodatavals:          (0.0, 0.0, 0.0, 0.0, 0.0)
    _FillValue:          0.0
    ...                  ...
    resampling:          nearest
    AREA_OR_POINT:       Area
   

# Coregister a planet to a landsat image


In [6]:
target_path = r"20240407_100338_88_2439_3B_AnalyticMS_clip.tif"
ref_path = r"ID_mqq51_sample_int3_new_block.tif"

with gw.open(target_path) as target, gw.open(
    ref_path
) as reference:
    target_shifted = gw.coregister(
        target=target,
        reference=reference,
        align_grids = True,
        resamp_alg_deshift='nearest',
        resamp_alg_calc='cubic',
        q=False, #quiet mode (default: False)
        CPUs=1, #  number of CPUs to use during pixel grid equalization (default: None, which means ‘all CPUs available’)
    )
    print(target_shifted)
    gw.save(target_shifted, 'coregistered_20240407_100338_88_2439_3B_AnalyticMS_clip.tif', compress='lzw', num_workers=8)

Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00
Polygonize progress     |===================-------------------------------| 38.0% Complete  => 0:00:00

Calculating footprint polygon and actual data corner coordinates for reference image...
Bounding box of calculated footprint for reference image:
	(662617.5, 5922562.5, 667087.5, 5927032.5)
Calculating footprint polygon and actual data corner coordinates for image to be shifted...


Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00


Bounding box of calculated footprint for image to be shifted:
	(662601.0, 5922564.0, 667119.0, 5927034.0)
Matching window position (X,Y): 664856.2255667822/5924807.507308026
Detected integer shifts (X/Y):                            0/0
Detected subpixel shifts (X/Y):                           0.2096289671243826/0.44645149890756547
Calculated total shifts in fft pixel units (X/Y):         0.2096289671243826/0.44645149890756547
Calculated total shifts in reference pixel units (X/Y):   0.2096289671243826/0.44645149890756547
Calculated total shifts in target pixel units (X/Y):      1.048144835621913/2.2322574945378273
Calculated map shifts (X,Y):				  3.1444345068885013/-6.696772483177483
Calculated absolute shift vector length in map units:     7.3982585829068075
Calculated angle of shift vector in degrees from North:   334.84784268936664
Original map info: ['UTM', 1.0, 1.0, 662601.0, 5927034.0, 3.0, 3.0, 31, 'North', 'WGS-84']
Updated map info:  ['UTM', 1.0, 1.0, '662604.1444345069', '59

Warping progress     |=========================-------------------------| 50.0% Complete  => 0:00:00

Image similarity within the matching window (SSIM before/after correction): 0.6136 => 0.6095
Estimated reliability of the calculated shifts:  80.8 %
Correcting geometric shifts...


Warping progress     |==================================================| 100.0% Complete  => 0:00:00
11:17:49:WARNING:755:io.save:The file coregistered_20240407_100338_88_2439_3B_AnalyticMS_clip.tif already exists.


<xarray.DataArray 'array-20f99b766dc89ef8d81baa5115451d89' (band: 4, y: 1489,
                                                            x: 1505)> Size: 72MB
dask.array<array, shape=(4, 1489, 1505), dtype=float64, chunksize=(4, 256, 256), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int32 16B 1 2 3 4
  * y        (y) float64 12kB 5.927e+06 5.927e+06 ... 5.923e+06 5.923e+06
  * x        (x) float64 12kB 6.626e+05 6.626e+05 ... 6.671e+05 6.671e+05
Attributes: (12/17)
    transform:           (3.0, 0.0, 662602.5, 0.0, -3.0, 5927032.5)
    crs:                 32631
    res:                 (3.0, 3.0)
    is_tiled:            1
    nodatavals:          (0.0, 0.0, 0.0, 0.0)
    _FillValue:          0.0
    ...                  ...
    AREA_OR_POINT:       Area
    TIFFTAG_DATETIME:    2024:04:07 10:03:38
    _data_are_separate:  0
    _data_are_stacked:   0
    x_shift_px:          1.048144835621913
    y_shift_px:          2.2322574945378273
